<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_4_SLSTR_bands_imagery.ipynb"><< SLSTR bands and imagery</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_6_SLSTR_SST.ipynb">Using SLSTR SST products >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SLSTR: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook, and will retrieve the data required here.
  - **<a href="./1_4_SLSTR_bands_imagery.ipynb" target="_blank">1_4_SLSTR_bands_imagery.ipynb</a>** if you are not familiar with the SLSTR grids at level-1B
    <br><br>**AND**<br><br>    
  - **<a href="./1_1a_SLSTR_data_access_Data_Store.ipynb" target="_blank">1_1a_SLSTR_data_access_Data_Store.ipynb</a>** if using the Data Store for data access
    <br><br>**OR**<br><br>
  - **<a href="./1_1b_SLSTR_data_access_HDA.ipynb" target="_blank">1_1b_SLSTR_data_access_HDA.ipynb</a>** if using WEkEO for data access
    
</div>
<hr>

# 1.5 SLSTR radiance and brightness temperature spectra
    
</div>

### Data used

| Product Description | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| Sentinel-3 SLSTR level-1B | EO:EUM:DAT:0411 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SL_1_RBT___NTC?query=SLSTR&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SL_1_RBT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASL_1_RBT___&initial=1" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* How to extract data from different wavebands in the SLSTR products from the associated netcdf files.
* How to extract the radiance and brightness temperature spectra over a specific area. 
* That the spectra retrieved by SLSTR vary depending on the temperature of water they represent.
* How to make interactive plots using the Bokeh library.

### Outline

While images can offer us a view into the spatial patterns in the ocean, it is the spectral variability that allows us to retrieve quantitative information on a given pixel, and so derive our key variables such as sea surface temperature. Furthermore, analysis of the spectral signal allows us to perform checks on our data, flagging it for cloud, land and other quality indices. 

In this notebook we will take a closer look at the spectral characteristics of level-1B data from the SLSTR instruments aboard Sentinel-3. We will focus on the differences between radiance and brightness temperature, and how and when each are used.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Radiance and brightness temperature](#section1)
 2. [Reading band data from the xml manifest file](#section2)
 3. [Reading in the radiance and brightness temperature](#section3)
 4. [Plotting the radiance and brightness temperature](#section4)
 5. [Applying your knowledge](#section5)

<hr>

Let's begin by importing all the Python libraries we need for this notebook.

In [1]:
# library imports
import glob                                             # a library that aids in searching for files
import inspect                                          # a library that lets us query function source code
import numpy as np                                      # a library that provides support for array-based mathematics
import os                                               # a library that allows us access to basic operating system commands like making directories
import warnings                                         # a library that helps us manage warnings
import xarray as xr                                     # a library that supports the use of multi-dimensional arrays in Python
import xml.etree.ElementTree as ET                      # a library that helps us parse XML files
import pandas as pd                                     # a library that provides analysis tools for time-series and dataframes
from bokeh.io import output_notebook, show, export_png  # a library that provides Bokeh plotting support
from bokeh.plotting import figure                       # a library that provides Bokeh plotting support
from bokeh.models import HoverTool, Range1d, LinearAxis # a library that provides Bokeh plotting support
from bokeh.layouts import row                           # a library that provides Bokeh plotting support
import eumartools                                       # a EUMETSAT library that support working with Sentinel-3 products
warnings.filterwarnings('ignore')

Now lets initiate Bokeh, which will support our plotting at the end of this notebook.

In [2]:
output_notebook()

Loading BokehJS ...

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Radiance and brightness temperature
[Back to top](#TOC_TOP)

</div>

The radiometry bands on SLSTR can be split into two types; those the measure top of atmosphere radiance and those that measure brightness temperature. The radiance bands, S1 to S6, derive their signal from the reflectance of visible and short wave infrared radiation from the sun from cloud, atmospheric aerosols, and the land and ocean surface. Consquently, they do not provide information at night when there is no solar radiation to reflect. From an oceanography perspective, these bands are useful in helping us to properly screen our data for pixels that may be contaminated with cloud etc.

In contrast, the thermal infrared bands (TIR), S7 - S9, measure brightness temperature. Brightness temperature (BT) is defined as the temperature at which a blackbody, in thermal equilibrium with its surroundings, would have to be in order to duplicate the observed intensity of an object at a given frequency. It allows us to relate the intensity of the emitted radiation and the temperature of an object. We use the information from these bands to derive sea surface temperature (SST).

*Note: in the day we are unable to use the S7 BT channel as it is contaminated with solar radiation. More information on this can be found in the next notebook*

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Reading band data from the xml manifest file
[Back to top](#TOC_TOP)

</div>

As we have gridded fields for each band, we can view them as spectra for a given pixel. In order to do this, we should first read the manifest file for our chosen product. This will provide us with necessary information on the bands, themselves. First lets supply our chosen product;

In [3]:
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_SL_1_RBT____20220209T225750_20220209T230050_20220211T073550_0179_082_001_3600_MAR_O_NT_004.SEN3')

Next, we will set up a simple dictionary that will allow us to store all the required data on band name, type, width etc.

In [4]:
# set up data dictionary
data = {'band_type': [],
        'band_names': [],
        'band_centres': [],
        'band_widths': [],
        'means': [],
        'uppers': [],
        'lowers': []}

Now we can read the required information from the XML file into our dictionary. Note that in each case we are reading the data as a list that is stored in each dictionary key (e.g. `band_names`).

In [5]:
# Read XML file; there is now a function for this print(inspect.getsource(eumartools.read_manifest))
tree = ET.parse(os.path.join(SAFE_directory,'xfdumanifest.xml'))
root = tree.getroot()
match = {"sentinel3":"http://www.esa.int/safe/sentinel/sentinel-3/1.0"}
data["band_names"] = [item.attrib["name"] for item in root.findall('.//sentinel3:band', match)]
data["band_centres"] = [float(item.text) for item in root.findall('.//sentinel3:centralWavelength', match)]
data["band_widths"] = [float(item.text) for item in root.findall('.//sentinel3:bandwidth', match)]

We are going to prune our list a bit as we do not want to retain the fire channels, which are read in at the end.

In [6]:
# remove the fire channels
for item in data:
    data[item] = data[item][0:9]

And finally we are going to define our band type based on the wavelength of the band centre. We will use 3000 nm as our threshold value, above which we assume that we have a brightness temperature.

In [7]:
for band_centre in data['band_centres']:
    if band_centre > 3000:
        data['band_type'].append('BT')
    else:
        data['band_type'].append('radiance')

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Reading in the radiance and brightness temperature data
[Back to top](#TOC_TOP)

</div>

Now we are ready to read in our data from the various files in the SAFE-format folder for this product. We will first define a small area to extract our spectrum over. In addition, we will reduce the size of our grid by a factor of 5 in each direction to help us manage the amount of data we work with.

In [8]:
# makes area average
lons = [151.0, 152.0,  152.0, 151.0, 151.0]
lats = [-36.0, -36.0, -35.0,  -35.0, -36.0]

# for minimap plotting only
grid_reduce = 5

Before we read in the radiance and brightness temperatures, we need to read in their respective grids. However, these are provided on different grids, so we need to read in two grids (see the **<a href="./1_4_SLSTR_bands_imagery.ipynb">1_4_SLSTR_bands_imagery</a>** notebook for more information)

In [9]:
# read the radiance nadir grid for the a-stripe
geo_radiance_file = os.path.join(SAFE_directory,'geodetic_an.nc')
geo_radiance_vars = xr.open_dataset(geo_radiance_file)
lon_radiance = geo_radiance_vars.longitude_an.data[::grid_reduce, ::grid_reduce]
lat_radiance = geo_radiance_vars.latitude_an.data[::grid_reduce, ::grid_reduce]
geo_radiance_vars.close()

# read the BT nadir grid
geo_BT_file = os.path.join(SAFE_directory,'geodetic_in.nc')
geo_BT_vars = xr.open_dataset(geo_BT_file)
lon_BT = geo_BT_vars.longitude_in.data[::grid_reduce, ::grid_reduce]
lat_BT = geo_BT_vars.latitude_in.data[::grid_reduce, ::grid_reduce]
geo_BT_vars.close()

We are now going to use a quick routine to subset this data, spatially. To see more information about how this works you can write `print(inspect.getsource(eumartools.subset_image))` into a new cell. This routine provides a mask that we will use to remove the data we are not interested in, allowing us to the take the mean of what remains.

In [10]:
# extract our data of interest 
ex, ey, mask = eumartools.subset_image(lon_radiance, lat_radiance, lons, lats)
mask_radiance = mask.T

ex, ey, mask = eumartools.subset_image(lon_BT, lat_BT, lons, lats)
mask_BT = mask.T

Now we can read in the data from all of our radiance channels. The code cell below will iterate through all of the radiance and BT channels we have specified above, apply our mask and take the mean to give us an average over our area for each channel.

In [11]:
# read everything we need from netcdf, open all the files at once
band_vars_radiance = xr.open_mfdataset(glob.glob(os.path.join(SAFE_directory,'*radiance_an.nc')))
band_vars_BT = xr.open_mfdataset(glob.glob(os.path.join(SAFE_directory,'*BT_in.nc')))

for band_vars,mask in zip([band_vars_radiance, band_vars_BT], [mask_radiance, mask_BT]):
    for band_var in band_vars:
        if "radiance_an" in band_var or "BT_in" in band_var and not "F" in band_var:
            print(f"Reading:  {band_var}")
            var = band_vars[band_var].data[::grid_reduce, ::grid_reduce]*mask
            data["means"].append(np.array(np.nanmean(var)))
            data["uppers"].append(np.array(np.nanmean(var) + np.nanstd(var)))
            data["lowers"].append(np.array(np.nanmean(var) - np.nanstd(var)))
            if "S7_BT_in" in band_var:
                # this is just to build our quick minimap
                minimap = band_vars[band_var].data[::grid_reduce, ::grid_reduce]

band_vars_radiance.close()
band_vars_BT.close()

Reading:  S1_radiance_an
Reading:  S2_radiance_an
Reading:  S3_radiance_an
Reading:  S4_radiance_an
Reading:  S5_radiance_an
Reading:  S6_radiance_an
Reading:  S7_BT_in
Reading:  S8_BT_in
Reading:  S9_BT_in


Finally, we will will convert our dictionary into a `dataframe`. This is a convenient format for working with series of data.

In [12]:
# put everything in Pandas dataframes for convenience
data["low_band"]  = [a-b for a,b in zip(data["band_centres"], data["band_widths"])]
data["high_band"] = [a+b for a,b in zip(data["band_centres"], data["band_widths"])]

data_radiances = [ { key : data[key][idx] for key in data.keys() }  
                  for idx, x in enumerate(data["band_type"]) if x == "radiance" ]
data_BT = [ { key : data[key][idx] for key in data.keys() }  
           for idx, x in enumerate(data["band_type"]) if x == "BT" ]

rad_df = pd.DataFrame(data_radiances)
BT_df = pd.DataFrame(data_BT)

<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Plotting the radiance and reflectance spectra
[Back to top](#TOC_TOP)

</div>

Now we are ready to plot our data. These cells look quite complex, as plotting often requires a degree of individual 'tinkering'. The next cell creates out "mini-map" inset, which shows us where our extraction region is. It creates a grayscale image of the S7 channel.

In [13]:
# build minimap
rgb = np.dstack((minimap, minimap, minimap))
                     
rgb = eumartools.normalise_image(rgb, unhitch='True')
rgb = eumartools.truncate_image(rgb)
rgb = eumartools.histogram_image(rgb, nbins=512)

# make image array
img = np.empty((np.shape(rgb)[0], np.shape(rgb)[1]), dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((np.shape(rgb)[0], np.shape(rgb)[1], 4))
for ii in range(np.shape(rgb)[-1]):
    view[:, :, ii] = rgb[:,:,ii]*255

# add alpha
view[:, :, 3] = 255

The final cell creates an interactive plot of the average radiance and brightness temperature (BT) spectra across our extraction region. The red and blue traces are associated with the Y-axes of the same colour. You can clearly see the differences in scale and units between the radiance and BT channels.

In [14]:
# create a new plot
hover = HoverTool(tooltips=[("(wavelength, rad/BT)", "($x, $y)")])

# spectra
p = figure(plot_width=750, plot_height=600,
           title="Co-located SLSTR radiances & brightness temperatures over subset region",
           y_range=(0, max(rad_df["uppers"])*1.1),
           tools=[hover, 'pan', 'box_zoom', 'save', 'reset'],
           toolbar_location="below")

p.title.align = 'center'
p.title.text_font_size = '14pt'
p.xaxis.axis_label = r"$$ \text{Wavelength } [nm] $$"
p.xaxis.axis_label_text_font_size = "14pt"
p.yaxis.axis_label = r"$$\color{navy} \text{TOA radiance } [mW m^{-2} sr^{-1} nm^{-1}] $$"
p.add_layout(LinearAxis(y_range_name="BT", axis_label=r"$$\color{firebrick} \text{TOA brightness temperature } [K] $$"),
             'right')
p.extra_y_ranges = {"BT": Range1d(start=min(BT_df["lowers"])*0.9, end=max(BT_df["uppers"])*1.1)}

for axis, sr, c, nm in zip(["default", "BT"], [rad_df, BT_df], ["navy", "firebrick"], ["radiance", "BT"]):
    p.varea(source=sr, x="band_centres", y1="lowers", y2="uppers", alpha=0.5, color=c, y_range_name=axis)
    p.line(source=sr, x="band_centres", y="means", line_width=2, color=c, y_range_name=axis, legend_label=nm)
    p.circle(source=sr, x="band_centres", y="means", line_color="black", fill_color=c, size=4, y_range_name=axis)
    p.segment(source=sr, x0="low_band", y0="means", x1="high_band", y1="means", line_color="black", y_range_name=axis)
    p.yaxis.axis_label_text_color = c
    p.yaxis.axis_label_text_font_size = "14pt"
        
p.legend.location = "top_right"
p.legend.title = 'Spectra'
p.legend.title_text_font_style = "bold"
p.legend.title_text_font_size = "14pt"

# minimap
m = figure(plot_width=200, plot_height=200, toolbar_location="below")
m.image_rgba(image=[np.flipud(img)], x=0, y=0, dw=np.shape(rgb)[0], dh=np.shape(rgb)[1])
m.line(x=ex, y=[np.shape(img)[1] - i for i in ey], color="firebrick")
m.axis.visible = False ; m.xgrid.visible = False ; m.ygrid.visible = False

show(row(p, m))

<div class="alert alert-danger" role="alert">

## <a id='section5'></a>5. Applying your knowledge
[Back to top](#TOC_TOP)

</div>

### What to try next?

* Investigate different areas in this scene; how does the BT spectrum look in different places?
* Compare BT and radiance spectra over land and over water, what do you notice?
* Trying looking at other images to compare how spectra look in the day and at night (this is a daytime image).

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_4_SLSTR_bands_imagery.ipynb"><< SLSTR bands and imagery</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_6_SLSTR_SST.ipynb">Using SLSTR SST products >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>